In [1]:
# Import libraries
import py_entitymatching as em
import pandas as pd
import os, sys

# Matching two tables typically consists of the following three steps:
** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# 1. Loading the input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [2]:
#dblp_dataset_path = os.sep.join(['DBLP_ACM', 'DBLP_cleaned.csv'])
#acm_dataset_path = os.sep.join(['DBLP_ACM', 'ACM_cleaned.csv'])

datasets_dir = os.getcwd() + os.sep + 'data\\csv\\Abt-Buy'

abt_dataset_path = datasets_dir + os.sep + 'abtProfiles.csv'
buy_dataset_path = datasets_dir + os.sep + 'buyProfiles.csv'

In [3]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(abt_dataset_path, encoding = "ISO-8859-1", key='id')
B = em.read_csv_metadata(buy_dataset_path, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 1076
Number of tuples in B: 1076
Number of tuples in A X B (i.e the cartesian product): 1157776


In [5]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

In [6]:
# If the tables are large we can downsample the tables like this
# A1, B1 = em.down_sample(A, B, 200, 1, show_progress=False)
# len(A1), len(B1)

# 2. Blocking to create candidate tuple pairs
Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.

Magellan provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. Refer to [api reference] for more details. The user can mix and match these blockers to form a blocking sequence applied to input tables.



In [75]:
ob = em.OverlapBlocker()
OB = ob.block_tables(A, B, 'name', 'name', l_output_attrs=['description', 'name', 'price'],
                   r_output_attrs=['description', 'name', 'price'])

C:\Users\DELL\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
C:\Users\DELL\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


## Attribute Equivalence Blocker

In [21]:
# Create attribute equivalence blocker on Price 
ab = em.AttrEquivalenceBlocker()
# Block tables using 'year' attribute: same year then include in the canidate set
AE = ab.block_tables(A, B, 'price', 'price', 
                   l_output_attrs=['description', 'name', 'price'],
                   r_output_attrs=['description', 'name', 'price']
                   )

In [23]:
AE.head()

,_id,ltable_id,rtable_id,ltable_description,ltable_name,ltable_price,rtable_description,rtable_name,rtable_price
0,0,1,674,Bose Acoustimass 5 Series III Speaker System - AM53BK/ 2 Dual Cube Speakers With Two 2-1/2' Wide...,Bose Acoustimass 5 Series III Speaker System - AM53BK,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
1,1,281,674,Tech Craft Dark Cherry Veneto Series TV Stand - SWP60/ 60' Wide Credenza For Flat Panel TV?s And...,Tech Craft Dark Cherry Veneto Series TV Stand - SWP60,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
2,2,337,674,Tech Craft Veneto Series Black TV Stand - ABS60BK/ Supports Up To A 60' Flat Panel TV/ Molded To...,Tech Craft Veneto Series Black TV Stand - ABS60BK,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
3,3,373,674,Microsoft Office Standard 2007 - 02107746/ Create Documents Faster More Easily And More Intuitiv...,Microsoft Office Standard 2007 - 02107746,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
4,4,472,674,Yamaha 5.1 Channel Home Theater In A Box System In Black - YHT390BK/ New Scene Compressed Music ...,Yamaha 5.1 Channel Home Theater In A Box System In Black - YHT390BK,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0


## Rule-base Blocker

In [9]:
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)
block_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x0000021BCFD3DEE0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x0000021BE15A1550>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x0000021BE1606160>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x0000021BE16065E0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,description_description_jac_qgm_3_qgm_3,description,description,qgm_3,qgm_3,jaccard,<function description_description_jac_qgm_3_qgm_3 at 0x0000021BE1606670>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,description_description_cos_dlm_dc0_dlm_dc0,description,description,dlm_dc0,dlm_dc0,cosine,<function description_description_cos_dlm_dc0_dlm_dc0 at 0x0000021BE1606700>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x0000021BE1606790>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x0000021BE1606820>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x0000021BE16068B0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,name_name_lev_dist,name,name,None,None,lev_dist,<function name_name_lev_dist at 0x0000021BE1606940>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [10]:
em._block_c['corres']

[('id', 'id'),
 ('description', 'description'),
 ('name', 'name'),
 ('entityUrl', 'entityUrl'),
 ('price', 'price')]

In [63]:
rb = em.RuleBasedBlocker()
# Add rule : block tuples if name_name_lev(ltuple, rtuple) < 0.4
# rb.add_rule(['name_name_cos_dlm_dc0_dlm_dc0(ltuple, rtuple) < 0.3'], block_f)
rb.add_rule(['name_name_mel(ltuple, rtuple) < 0.7'], block_f)

'_rule_0'

In [64]:
R1 = rb.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'], show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:34


In [65]:
R1.head()

,_id,ltable_id,rtable_id,ltable_name,rtable_name
0,0,0,18,Sony Turntable - PSLX350H,Sony VAIO Notebook and AC Adapter Case - VGPAMC2
1,1,0,20,Sony Turntable - PSLX350H,Sony Notebook and AC Adapter Cases - VGPAMC3
2,2,0,23,Sony Turntable - PSLX350H,Sony High Definition Mini DV Cassette - DSL-190-B
3,3,0,32,Sony Turntable - PSLX350H,Sony MiniDV Head Cleaner - DVM12CLD
4,4,0,33,Sony Turntable - PSLX350H,Sony SB-V40S A/V Selector - SBV40S


In [66]:
len(R1)

55016

## Black Box Blocker

In [28]:
# Apply black box blocker
# Create black box blocker
bb = em.BlackBoxBlocker()

In [29]:
import re

def extract_words(sentence):
    sentence = re.sub(r'[^\w\s]', '', sentence)

    words = sentence.split()

    return words

# BLack Box Function
def my_function(x, y):
    # x, y will be of type pandas series
    
    # get name attribute
    x_name = x['name']
    y_name = y['name']
    # get last names
    x_code = extract_words(x_name)[-1]
    y_extract = extract_words(y_name)
    # check if last names match
    if x_code in y_extract:
        return False
    else:
        return True

In [37]:
bb.set_black_box_function(my_function)

In [38]:
# F = bb.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'], show_progress=False)
BB = bb.block_candset(R1, show_progress=False)

In [39]:
# Check the number of rows in C1
BB.head()

,_id,ltable_id,rtable_id,ltable_name,rtable_name
14,14,1023,0,Linksys EtherFast 8-Port 10/100 Switch - EZXS88W,Linksys EtherFast EZXS88W Ethernet Switch - EZXS88W
21,21,1022,1,Linksys EtherFast10/100 5-Port Auto-Sensing Switch - EZXS55W,Linksys EtherFast EZXS55W Ethernet Switch
29,29,434,2,Netgear ProSafe 5 Port 10/100 Desktop Switch - FS105,Netgear ProSafe FS105 Ethernet Switch - FS105NA
35,35,214,3,Belkin F3H982-10 Pro Series High Integrity 10 Feet Monitor Cable - F3H98210,Belkin Pro Series High Integrity VGA/SVGA Monitor Extension Cable - F3H982-10
56,56,825,5,LaCie USB 2.0 Floppy Disk Drive - 706018,LaCie Pocket Floppy Disk Drive - 706018


# Combine Block

In [76]:
G = em.combine_blocker_outputs_via_union([AE, R1, BB, OB])

In [77]:
G

,_id,ltable_id,rtable_id,ltable_description,ltable_name,ltable_price,rtable_description,rtable_name,rtable_price
0,0,0,18,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,VAIO NEOPRENE NOTEBOOK & AC ADAPTER CASE UP TO 17 IN LCD,Sony VAIO Notebook and AC Adapter Case - VGPAMC2,NaN
1,1,0,20,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,Clam Shell - Neoprene,Sony Notebook and AC Adapter Cases - VGPAMC3,12.99
2,2,0,23,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,Sony DVM-63 HD High-Definition miniDV Videocassette,Sony High Definition Mini DV Cassette - DSL-190-B,NaN
3,3,0,28,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,NaN,SONY 7-11719-70670-0 PS2 8 MB Memory Card 2-pk - 97067,NaN
4,4,0,32,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,Head Cleaner,Sony MiniDV Head Cleaner - DVM12CLD,7.95
...,...,...,...,...,...,...,...,...,...
161191,161191,1075,783,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,1.6 cu.ft. Over the Range Microwave,NaN
161192,161192,1075,784,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,LG 2.0 cu.ft. Over the Range Microwave Oven,NaN
161193,161193,1075,785,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,LG 2.0 cu.ft. Over the Range Microwave Oven,NaN
161194,161194,1075,786,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,LG 2.0 cu.ft. Over the Range Microwave Oven,NaN


The number of tuple pairs considered for matching is reduced, but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in Magellan as follows:

In [42]:
# Debug blocker output
dbg = em.debug_blocker(G, A, B, output_size=200)

In [43]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head()

,_id,ltable_id,rtable_id,ltable_description,ltable_name,rtable_description,rtable_name
0,0,568,680,Samsung 46' Series 6 LCD Black Flat Panel HDTV - LN46A650/ Full 1080p High Definition Resolution...,Samsung 46' Series 6 LCD Black Flat Panel HDTV - LN46A650,Samsung LN19A450 - 19' Widescreen 720p LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response T...,Samsung LN19A450 19' LCD TV
1,1,569,680,Samsung 52' Series 6 LCD Black Flat Panel HDTV - LN52A650/ Full 1080p High Definition Resolution...,Samsung 52' Series 6 LCD Black Flat Panel HDTV - LN52A650,Samsung LN19A450 - 19' Widescreen 720p LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response T...,Samsung LN19A450 19' LCD TV
2,2,800,586,Toshiba Black DVD/VCR Combinaton Player - SDV296/ Progressive Scan DVD Player/ One Touch Recordi...,Toshiba Black DVD/VCR Combinaton Player - SDV296,DVD+RW DVD-RW DVD-RAM DVD-R DVD+R CD-RW VHS Secure Digital (SD) Card - DVD Video CD-DA MP3 JPEG ...,Panasonic DMR-EA38VK DVD/VCR Combo
3,3,608,656,Samsung 42' Black Flat Panel Series 4 Plasma HDTV - PN42A450P/ 1024 x 768p Resolution/ 1000000:1...,Samsung 42' Black Flat Panel Series 4 Plasma HDTV - PN42A450P,Samsung PN50A550 - 50' Widescreen 1080p Plasma HDTV - 1000000:1 Dynamic Contrast Ratio - Piano B...,Samsung PN50A550 50' Plasma TV
4,4,736,744,Samsung Black Blu-ray Player - BDP1500/ 1080p With 24Hz Video Output/ x.v.Color Connectivity (HD...,Samsung Black Blu-ray Player - BDP1500,BD Player 5.1 Speakers - 1 Disc(s) - Progressive Scan - 1250W RMS - Dolby Digital Dolby TrueHD D...,Panasonic SC-BT100 Home Theater System - SC-BT100P-K


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the conference papers to match the author names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.

# 3. Matching tuple pairs in the candidate set
In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.

This typically involves the following five steps:

1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set
## 3.1 Creating a set of learning-based matchers

First, we randomly sample 450 tuple pairs for labeling purposes.

In [144]:
# Sample candidate set
# S = em.sample_table(C, 2000)

In [78]:
# Read Perfect Matching
matching_dataset_path = datasets_dir + os.sep + 'abtBuyIdDuplicates.csv'
M = em.read_csv_metadata(matching_dataset_path, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [84]:
len(M)

1076

In [81]:
# Label for data from Matching
G['label'] = 0
for index, row in M.iterrows():
    ltable_id = row['entityId1']
    rtable_id = row['entityId2']
    
    G.loc[(G['ltable_id'] == ltable_id) & (G['rtable_id'] == rtable_id), 'label'] = 1

## 3.2 Splitting the labeled data into development and evaluation set
In this step, we split the labeled data into two sets: development and evaluation. Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [85]:
# Split Matched data
matched = G.loc[G['label'] == 1]

em.set_key(matched,'_id')
em.set_fk_ltable(matched,'ltable_id')
em.set_fk_rtable(matched,'rtable_id')
em.set_ltable(matched, A)
em.set_rtable(matched, B)

matched_train_test = em.split_train_test(matched, train_proportion=0.7)
matched_development =  matched_train_test['train']
matched_evaluation = matched_train_test['test']

In [91]:
# Split Un-Matched data
unmatched = G.loc[G['label'] == 0].head(1000)

em.set_key(unmatched,'_id')
em.set_fk_ltable(unmatched,'ltable_id')
em.set_fk_rtable(unmatched,'rtable_id')
em.set_ltable(unmatched, A)
em.set_rtable(unmatched, B)

unmatched_train_test = em.split_train_test(unmatched, train_proportion=0.7)
unmatched_development =  unmatched_train_test['train']
unmatched_evaluation = unmatched_train_test['test']

In [93]:
# Combine 
development = pd.concat([matched_development, unmatched_development])
evaluation = pd.concat([matched_evaluation, unmatched_evaluation])

In [95]:
em.set_key(development,'_id')
em.set_fk_ltable(development,'ltable_id')
em.set_fk_rtable(development,'rtable_id')
em.set_ltable(development, A)
em.set_rtable(development, B)

em.set_key(evaluation,'_id')
em.set_fk_ltable(evaluation,'ltable_id')
em.set_fk_rtable(evaluation,'rtable_id')
em.set_ltable(evaluation, A)
em.set_rtable(evaluation, B)

True

## 3.3 Select the best learning-based matcher
Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Extracting feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation
5. Debugging the matcher (and possibly repeat the above steps)

### 3.3.1 Creating a set of learning-based matchers
First, we need to create a set of learning-based matchers. The following matchers are supported in Magellan: (1) decision tree, (2) random forest, (3) naive bayes, (4) svm, (5) logistic regression, and (6) linear regression.

In [96]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree')
svm = em.SVMMatcher(name='SVM')
rf = em.RFMatcher(name='RF')
nb = em.NBMatcher(name='NB')
lg = em.LogRegMatcher(name='LogReg')
ln = em.LinRegMatcher(name='LinReg')

### 3.3.2 Creating features
Next, we need to create a set of features for the development set. Magellan provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [132]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [133]:
# List the names of the features generated
feature_table['feature_name']

0                                       id_id_exm
1                                       id_id_anm
2                                  id_id_lev_dist
3                                   id_id_lev_sim
4         description_description_jac_qgm_3_qgm_3
5     description_description_cos_dlm_dc0_dlm_dc0
6                       name_name_jac_qgm_3_qgm_3
7                   name_name_cos_dlm_dc0_dlm_dc0
8                                   name_name_mel
9                              name_name_lev_dist
10                              name_name_lev_sim
11                        entityUrl_entityUrl_exm
12                        entityUrl_entityUrl_anm
13                   entityUrl_entityUrl_lev_dist
14                    entityUrl_entityUrl_lev_sim
15                                price_price_exm
16                                price_price_anm
17                           price_price_lev_dist
18                            price_price_lev_sim
Name: feature_name, dtype: object

In [134]:
feature_subset_iter1 = feature_table.drop([0,1,2,3,11,12,13,14], axis=0)

In [135]:
feature_subset_iter1['feature_name']

4         description_description_jac_qgm_3_qgm_3
5     description_description_cos_dlm_dc0_dlm_dc0
6                       name_name_jac_qgm_3_qgm_3
7                   name_name_cos_dlm_dc0_dlm_dc0
8                                   name_name_mel
9                              name_name_lev_dist
10                              name_name_lev_sim
15                                price_price_exm
16                                price_price_anm
17                           price_price_lev_dist
18                            price_price_lev_sim
Name: feature_name, dtype: object

### 3.3.3 Extracting feature vectors
In this step, we extract feature vectors using the development set and the created features.

In [154]:
# Extract feature vectors
feature_vectors_dev = em.extract_feature_vecs(development, 
                            feature_table=feature_subset_iter1, 
                            attrs_after='label')


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [171]:
# Display first few rows
feature_vectors_dev.head(3)

,_id,ltable_id,rtable_id,description_description_jac_qgm_3_qgm_3,description_description_cos_dlm_dc0_dlm_dc0,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,label
121260,121260,816,789,0.070671,0.251976,0.367647,0.629941,0.681338,23.0,0.549020,0.044643,0.595846,3.532738,0.36909,1
52142,52142,428,223,0.064997,0.151095,0.347826,0.338062,0.623189,38.0,0.254902,0.044643,0.595846,3.532738,0.36909,1
3684,3684,32,50,0.097902,0.356348,0.613636,0.670820,0.927451,23.0,0.323529,0.044643,0.595846,3.532738,0.36909,1


In [156]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_dev))

True

We observe that the extracted feature vectors contain missing values. We have to impute the missing values for the learning-based matchers to fit the model correctly. For the purposes of this guide, we impute the missing value in a column with the mean of the values in that column.

In [157]:
# Impute feature vectors with the mean of the column values.
feature_vectors_dev = em.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\matcher\matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


### 3.3.4 Selecting the best matcher using cross-validation
Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' metric to select the best matcher.

In [158]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=10,
        target_attr='label', 
        metric_to_select_matcher='precision',
        random_state=0)

In [159]:
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.955745,0.952459,0.955888
1,RF,0.977707,0.959269,0.970977
2,SVM,0.721323,0.515333,0.600406
3,NB,0.975028,0.897868,0.934505
4,LogReg,0.982434,0.922176,0.951109
5,LinReg,0.984850,0.884845,0.931885


In [160]:
result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x0000021BE553CA30>,10,0.958904,0.916667,0.962963,0.985714,0.957746,0.954545,0.964286,0.926471,0.944444,0.985714,0.955745
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x0000021BE553C400>,10,0.972222,0.970588,0.987342,0.986111,1.000000,0.988372,1.000000,0.942857,0.957746,0.971831,0.977707
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x0000021BE5420A90>,10,0.705882,0.644068,0.767857,0.694915,0.714286,0.803571,0.677966,0.760000,0.700000,0.744681,0.721323
3,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x0000021BE553C0A0>,10,1.000000,0.955224,0.987013,0.985507,0.968750,1.000000,0.973684,0.969697,0.941176,0.969231,0.975028
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x0000021BE553C700>,10,1.000000,0.957143,1.000000,1.000000,1.000000,1.000000,0.986842,0.954545,0.955224,0.970588,0.982434
5,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x0000021BE553CC70>,10,0.967742,1.000000,0.986301,1.000000,0.984375,1.000000,1.000000,0.983607,0.926471,1.000000,0.984850


### 3.3.5 Debugging matcher
We observe that the best matcher is not getting us to the precision that we expect (i.e > 95%). We debug the matcher to see what might be wrong.

To do this, first we split the feature vectors into train and test.

In [161]:
# # Split feature vectors into train and test
train_test = em.split_train_test(feature_vectors_dev, train_proportion=0.5)
train = train_test['train']
test = train_test['test']

In [162]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        target_attr='label')

From the GUI, we observe that using only 'year' related features result in a lot of false positives. So we decide to use all the features in the feature table (which had author, title and year related features).

In [148]:
# Select all features from the feature table
# feature_subset_iter2 = feature_table

In [149]:
# Get new set of features
# feature_vectors_dev = em.extract_feature_vecs(development, feature_table=feature_subset_iter2, attrs_after='label')

In [150]:
# Check if imputation is required
# any(pd.isnull(feature_vectors_dev))

In [151]:
# Impute feature vectors
# feature_vectors_dev = em.impute_table(feature_vectors_dev, 
#                 exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
#                 strategy='mean')

In [230]:
# Apply cross validation to find if there is a better matcher
# result = em.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
#         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
#         k=10,
#         target_attr='label', 
#         metric_to_select_matcher='precision',
#         random_state=0)

In [163]:
# result['cv_stats']

In [164]:
# result['drill_down_cv_stats']['precision']

Now, observe the best matcher is achieving the expected precision and we can proceed on to evaluating the best matcher on the unseen data (the evaluation set).

## 3.4 Evaluating the matching output
Evaluating the matching outputs for the evaluation set typically involves the following four steps:

1. Extracting the feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches
### 3.4.1 Extracting the feature vectors
As before, we extract the feature vectors (using the updated feature table and the evaluation set) and impute it (if necessary).

In [177]:
# Get new set of features
feature_vectors_eval = em.extract_feature_vecs(evaluation, 
                                               feature_table=feature_subset_iter1, 
                                               attrs_after='label')

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [178]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_eval))

True

In [179]:
# Impute feature vectors
feature_vectors_eval = em.impute_table(feature_vectors_eval, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\matcher\matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


### 3.4.2 Training the matcher
Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [180]:
# Train using feature vectors from the development set
rf.fit(table=feature_vectors_dev, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

### 3.4.3 Predicting the matches
Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [181]:
feature_vectors_eval.shape

(622, 15)

In [182]:
# Predict M 
predictions = rf.predict(table=feature_vectors_eval, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

### 3.4.4 Evaluating the matching output
Finally, we evaluate the predicted outputs

In [183]:
# Evaluate the result
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 97.15% (307/316)
Recall : 95.34% (307/322)
F1 : 96.24%
False positives : 9 (out of 316 positive predictions)
False negatives : 15 (out of 306 negative predictions)
